## Import Required Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import randint
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.datasets import make_classification
from sklearn.preprocessing import binarize, LabelEncoder, MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2

from sklearn.ensemble import AdaBoostClassifier # Boost
from sklearn.tree import DecisionTreeClassifier # DTC
from sklearn.naive_bayes import GaussianNB      # NB
from sklearn.svm import SVC                     #SVC


from sklearn import metrics
from sklearn.metrics import ( accuracy_score, mean_squared_error,
            precision_recall_curve, classification_report, confusion_matrix )
from sklearn.model_selection import cross_val_score
from collections import Counter

import warnings
warnings.filterwarnings('ignore')


## Dataset

* <h3> Assigning identifiers to questionnaire questions </h3>

* **age**
* **gender**
* **country**
* **race**
* **mental_health**: does the individual currently have mental health disorder?
* **mental_health_diagnosed**: was the individual diagnosed from a professional for mental health disorder?
* **family_history:** Do you have a family history of mental illness?
* **treatment:** Have you sought treatment for a mental health condition?
* **self_employed:** Are you self-employed?
* **work_interfere:** If you have a mental health condition, do you feel that it interferes with your work?
* **no_employees:** How many employees does your company or organization have?
* **tech_company:** Is your employer primarily a tech company/organization?
* **benefits:** Does your employer provide mental health benefits?
* **care_options:** Do you know the options for mental health care your employer provides?
* **wellness_program:** Has your employer ever discussed mental health as part of an employee wellness program?
* **seek_help:** Does your employer provide resources to learn more about mental health issues and how to seek help?
* **anonymity:** Is your anonymity protected if you choose to take advantage of mental health or substance abuse treatment resources?
* **leave:** How easy is it for you to take medical leave for a mental health condition?
* **coworkers:** Would you be willing to discuss a mental health issue with your coworkers?
* **mental_importance**: how much importance does your employer place on mental health?
* **physical_importance**: how much importance does your employer place on physical health?
* **supervisor**: Would you feel comfortable discussing a mental health issue with your direct supervisor?
* **mental_health_interview**: Would you bring up your mental health with a potential employer in an interview?
* **mental_vs_physical**: Would you feel more comfortable talking to your coworkers about your physical health or your mental health?
* **obs_neg_response**: Have you observed or experienced an unsupportive or badly handled response to a mental health issue in your current or previous workplace?
* **neg_consequence_coworker**: Have your observations of how another individual who discussed a mental health issue made you less likely to reveal a mental health issue yourself in your current workplace 

* <h3> Load dataset </h3>

We proceed with 4 of the compatible years, based on the selected variables

In [2]:
osmi14 = data = pd.read_csv('./data_years_cleaned/OSMI_2014_cleaned.csv', na_values='NA')
osmi19 = data = pd.read_csv('./data_years_cleaned/OSMI_2019_cleaned.csv', na_values='NA')
osmi20 = data = pd.read_csv('./data_years_cleaned/OSMI_2020_cleaned.csv', na_values='NA')
osmi21 = data = pd.read_csv('./data_years_cleaned/OSMI_2021_cleaned.csv', na_values='NA')

* Concatenate dataset

In [3]:
data = pd.concat([osmi14, osmi19, osmi20, osmi21], ignore_index=True)
print(data.describe())
print(data.info())

                age  Unnamed: 0  physical_importance  mental_importance
count  1.922000e+03  663.000000           573.000000         573.000000
mean   5.202917e+07  131.318250             6.462478           5.069808
std    2.280990e+09   94.286827             2.285728           2.565644
min   -1.726000e+03    1.000000             0.000000           0.000000
25%    2.700000e+01   56.000000             5.000000           3.000000
50%    3.200000e+01  111.000000             7.000000           5.000000
75%    3.700000e+01  186.500000             8.000000           7.000000
max    1.000000e+11  352.000000            10.000000          10.000000
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1922 entries, 0 to 1921
Data columns (total 35 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Timestamp                  1259 non-null   object 
 1   age                        1922 non-null   int64  
 2   gender          

In [4]:
data.columns

Index(['Timestamp', 'age', 'gender', 'country', 'state', 'self_employed',
       'family_history', 'treatment', 'work_interfere', 'no_employees',
       'remote_work', 'tech_company', 'benefits', 'care_options',
       'wellness_program', 'seek_help', 'anonymity', 'leave',
       'mental_health_consequence', 'phys_health_consequence', 'coworkers',
       'supervisor', 'mental_health_interview', 'phys_health_interview',
       'mental_vs_physical', 'obs_consequence', 'comments', 'Unnamed: 0',
       'race', 'mental_health', 'mental_health_diagnosed', 'obs_neg_response',
       'physical_importance', 'mental_importance', 'neg_consequence_coworker'],
      dtype='object')

## Data Preprocessing

* Checking for missing values

In [5]:
total_missing = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total_missing, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)
print(missing_data)

                           Total   Percent
comments                    1758  0.914672
mental_health_diagnosed     1681  0.874610
race                        1605  0.835068
neg_consequence_coworker    1390  0.723205
mental_importance           1349  0.701873
physical_importance         1349  0.701873
obs_neg_response            1259  0.655047
mental_health               1259  0.655047
Unnamed: 0                  1259  0.655047
state                       1178  0.612903
mental_health_consequence    663  0.344953
obs_consequence              663  0.344953
phys_health_interview        663  0.344953
phys_health_consequence      663  0.344953
Timestamp                    663  0.344953
remote_work                  663  0.344953
work_interfere               264  0.137357
care_options                 149  0.077523
seek_help                     90  0.046826
benefits                      90  0.046826
no_employees                  90  0.046826
anonymity                     90  0.046826
mental_vs_p

* Drop comments, state, unnamed, country and Timestamp from features

In [6]:
data.drop(['comments'], axis= 1, inplace=True)
data.drop(['state'], axis= 1, inplace=True)
data.drop(['Timestamp'], axis= 1, inplace=True)
data.drop(['Unnamed: 0'], axis= 1, inplace=True)
data.drop(['country'], axis= 1, inplace=True)

In [7]:
data.isnull().sum().max()
data.head(5)

,age,gender,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,benefits,...,phys_health_interview,mental_vs_physical,obs_consequence,race,mental_health,mental_health_diagnosed,obs_neg_response,physical_importance,mental_importance,neg_consequence_coworker
0,37,Female,NaN,No,Yes,Often,Jun-25,No,Yes,Yes,...,Maybe,Yes,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,44,M,NaN,No,No,Rarely,More than 1000,No,No,Don't know,...,No,Don't know,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,32,Male,NaN,No,No,Rarely,Jun-25,No,Yes,No,...,Yes,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,31,Male,NaN,Yes,Yes,Often,26-100,No,Yes,No,...,Maybe,No,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,31,Male,NaN,No,No,Never,100-500,Yes,Yes,Yes,...,Yes,Don't know,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* Fill NaNs with default data types

In [8]:
# Assign default values for each data type:
fillInt = 0
#fillString = 'NaN'
fillFloat = 0.0

# Create lists by data type
intFeatures = ['age']
stringFeatures = ['gender', 'country', 'self_employed', 'family_history', 'treatment', 'work_interfere',
                 'no_employees', 'remote_work', 'tech_company', 'anonymity', 'leave', 'mental_health_consequence',
                 'phys_health_consequence', 'coworkers', 'supervisor', 'mental_health_interview', 'phys_health_interview',
                 'mental_vs_physical', 'obs_consequence', 'benefits', 'care_options', 'wellness_program',
                 'seek_help']
floatFeatures = []

# fill NaNs with default datatype:
for feature in data:
    if feature in intFeatures:
        data[feature] = data[feature].fillna(fillInt)
    elif feature in stringFeatures:
         data[feature] = data[feature].fillna(data[feature].mode().tolist()[0])
    elif feature in floatFeatures:
        data[feature] = data[feature].fillna(fillFloat)
    else:
        print(f'Error: Feature {feature} not recognized.')
data.head(100)

Error: Feature race not recognized.
Error: Feature mental_health not recognized.
Error: Feature mental_health_diagnosed not recognized.
Error: Feature obs_neg_response not recognized.
Error: Feature physical_importance not recognized.
Error: Feature mental_importance not recognized.
Error: Feature neg_consequence_coworker not recognized.


,age,gender,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,benefits,...,phys_health_interview,mental_vs_physical,obs_consequence,race,mental_health,mental_health_diagnosed,obs_neg_response,physical_importance,mental_importance,neg_consequence_coworker
0,37,Female,No,No,Yes,Often,Jun-25,No,Yes,Yes,...,Maybe,Yes,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,44,M,No,No,No,Rarely,More than 1000,No,No,Don't know,...,No,Don't know,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,32,Male,No,No,No,Rarely,Jun-25,No,Yes,No,...,Yes,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,31,Male,No,Yes,Yes,Often,26-100,No,Yes,No,...,Maybe,No,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,31,Male,No,No,No,Never,100-500,Yes,Yes,Yes,...,Yes,Don't know,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,29,male,No,No,No,Never,26-100,No,Yes,Don't know,...,Maybe,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,24,Male,No,Yes,No,Rarely,26-100,No,Yes,Don't know,...,Maybe,Don't know,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,31,male,No,Yes,Yes,Sometimes,Jun-25,No,Yes,Don't know,...,Yes,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,33,Cis Male,No,No,Yes,Sometimes,Jun-25,No,Yes,No,...,Maybe,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN


* Regroup gender into three levels: male, female and trans

In [9]:
gender = data['gender'].unique()
print(gender)

['Female' 'M' 'Male' 'male' 'female' 'm' 'Male-ish' 'maile' 'Trans-female'
 'Cis Female' 'F' 'something kinda male?' 'Cis Male' 'Woman' 'f' 'Mal'
 'Male (CIS)' 'queer/she/they' 'non-binary' 'Femake' 'woman' 'Make' 'Nah'
 'All' 'Enby' 'fluid' 'Genderqueer' 'Female ' 'Androgyne' 'Agender'
 'cis-female/femme' 'Guy (-ish) ^_^' 'male leaning androgynous' 'Male '
 'Man' 'Trans woman' 'msle' 'Neuter' 'Female (trans)' 'queer'
 'Female (cis)' 'Mail' 'cis male' 'A little about you' 'Malr' 'p' 'femail'
 'Cis Man' 'ostensibly male, unsure what that really means'
 'Let\'s keep it simple and say "male"' 'Non-binary' 'Identify as male'
 'Non binary' 'Masculine' 'Cishet male' 'None' 'Female-identified'
 'Nonbinary' 'agender' 'Questioning' 'I have a penis' 'rr' 'cis woman'
 'Agender trans woman' 'femmina' 'Trans man' 'man' 'masculino'
 'I am a Wookie' 'Trans non-binary/genderfluid' 'CIS Male'
 'Non-binary and gender fluid' 'Femile' 'mail' 'FEMALE' 'female, she/her'
 'mostly male' 'cisgender male' 'MAle

In [10]:
# gender groups:

male_str = ["male",'rr','mostly male','male/he/him','b','masculino','female, she/her', "m",'i have a penis', 'masculine','identify as male',"male-ish",'let\'s keep it simple and say "male"', "maile", "mal", "male (cis)", "make", "male ", "man","msle", "mail", "malr","cis man", "Cis Male", "cis male"]
trans_str = ["trans-female",'non-binary/agender','cis-het male','homem cis','afab non-binary','cisgender male','femile',"cis female",'i am a wookie','trans man','agender trans woman','cis woman','questioning','nonbinary','non-binary and gender fluid','trans non-binary/genderfluid','non binary', "something kinda male?", "queer/she/they", "non-binary","nah", "all", "enby", "fluid", "genderqueer", "androgyne", "agender", "male leaning androgynous", "guy (-ish) ^_^", "trans woman", "neuter", "female (trans)", "queer", "ostensibly male, unsure what that really means"]           
female_str = ['femmina','female-identified','none', 'cishet male',"f", "female", "woman", "femake", "female ","cis-female/femme", "female (cis)", "femail"]

for row, col in data.iterrows():

    if str.lower(col.gender) in male_str:
        data['gender'].replace(to_replace=col.gender, value='male', inplace=True)

    if str.lower(col.gender) in female_str:
        data['gender'].replace(to_replace=col.gender, value='female', inplace=True)

    if str.lower(col.gender) in trans_str:
        data['gender'].replace(to_replace=col.gender, value='trans', inplace=True)

#filter out unnecessary factors
stk_list = ['A little about you', 'p']
data = data[~data['gender'].isin(stk_list)]

print(data['gender'].unique())

['female' 'male' 'trans']


* Fill missing values

In [11]:
#complete missing age with mean
data['age'].fillna(data['age'].mean(), inplace = True)

# Fill with median() values < 18 and > 120
s = pd.Series(data['age'])
s[s<18] = data['age'].median()
data['age'] = s
s = pd.Series(data['age'])
s[s>120] = data['age'].median()
data['age'] = s

#Ranges of age
data['age_range'] = pd.cut(data['age'], [0,20,30,65,100], labels=["0-20", "21-30", "31-65", "66-100"], include_lowest=True)

* Drop new column, unnamed: 0

In [12]:
data.columns

Index(['age', 'gender', 'self_employed', 'family_history', 'treatment',
       'work_interfere', 'no_employees', 'remote_work', 'tech_company',
       'benefits', 'care_options', 'wellness_program', 'seek_help',
       'anonymity', 'leave', 'mental_health_consequence',
       'phys_health_consequence', 'coworkers', 'supervisor',
       'mental_health_interview', 'phys_health_interview',
       'mental_vs_physical', 'obs_consequence', 'race', 'mental_health',
       'mental_health_diagnosed', 'obs_neg_response', 'physical_importance',
       'mental_importance', 'neg_consequence_coworker', 'age_range'],
      dtype='object')

In [13]:
features = data.columns

for feature in features:
    print(f"======== UNIQUE FACTORS IN {feature} ======")
    print(data[feature].unique())
   

======== UNIQUE FACTORS IN age ======
[37 44 32 31 33 35 39 42 23 29 36 27 46 41 34 30 40 38 50 24 18 28 26 22
 19 25 45 21 43 56 60 54 55 48 20 57 58 47 62 51 65 49 53 61 72 52 64 63
 59]
======== UNIQUE FACTORS IN gender ======
['female' 'male' 'trans']
======== UNIQUE FACTORS IN self_employed ======
['No' 'Yes' False True]
======== UNIQUE FACTORS IN family_history ======
['No' 'Yes' "I don't know"]
======== UNIQUE FACTORS IN treatment ======
['Yes' 'No' False True]
======== UNIQUE FACTORS IN work_interfere ======
['Often' 'Rarely' 'Never' 'Sometimes' 'Not applicable to me']
======== UNIQUE FACTORS IN no_employees ======
['Jun-25' 'More than 1000' '26-100' '100-500' '01-May' '500-1000' '6-25'
 '1-5']
======== UNIQUE FACTORS IN remote_work ======
['No' 'Yes']
======== UNIQUE FACTORS IN tech_company ======
['Yes' 'No' True False]
======== UNIQUE FACTORS IN benefits ======
['Yes' "Don't know" 'No' "I don't know" 'Not eligible for coverage / NA']
======== UNIQUE FACTORS IN care_options =

* Replace all boolean values with "Yes" and "No"

In [14]:
features = data.columns

for feature in features:
    data[feature] = data[feature].replace(True, 'Yes')
    data[feature] = data[feature].replace(False, 'No')
    

In [15]:
features = data.columns

counter = 1
for feature in features:
    print(f" {counter} ======== UNIQUE FACTORS IN {feature} ======")
    print(data[feature].unique())
    counter += 1
    print("\n")

 1 ======== UNIQUE FACTORS IN age ======
[37 44 32 31 33 35 39 42 23 29 36 27 46 41 34 30 40 38 50 24 18 28 26 22
 19 25 45 21 43 56 60 54 55 48 20 57 58 47 62 51 65 49 53 61 72 52 64 63
 59]


 2 ======== UNIQUE FACTORS IN gender ======
['female' 'male' 'trans']


 3 ======== UNIQUE FACTORS IN self_employed ======
['No' 'Yes']


 4 ======== UNIQUE FACTORS IN family_history ======
['No' 'Yes' "I don't know"]


 5 ======== UNIQUE FACTORS IN treatment ======
['Yes' 'No']


 6 ======== UNIQUE FACTORS IN work_interfere ======
['Often' 'Rarely' 'Never' 'Sometimes' 'Not applicable to me']


 7 ======== UNIQUE FACTORS IN no_employees ======
['Jun-25' 'More than 1000' '26-100' '100-500' '01-May' '500-1000' '6-25'
 '1-5']


 8 ======== UNIQUE FACTORS IN remote_work ======
['No' 'Yes']


 9 ======== UNIQUE FACTORS IN tech_company ======
['Yes' 'No']


 10 ======== UNIQUE FACTORS IN benefits ======
['Yes' "Don't know" 'No' "I don't know" 'Not eligible for coverage / NA']


 11 ======== UNIQUE FAC

In [16]:
total_missing = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total_missing, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)
print(missing_data)

                           Total   Percent
mental_health_diagnosed     1679  0.874479
race                        1603  0.834896
neg_consequence_coworker    1388  0.722917
mental_importance           1347  0.701562
physical_importance         1347  0.701562
obs_neg_response            1257  0.654687
mental_health               1257  0.654687
age                            0  0.000000
phys_health_consequence        0  0.000000
obs_consequence                0  0.000000
mental_vs_physical             0  0.000000
phys_health_interview          0  0.000000
mental_health_interview        0  0.000000
supervisor                     0  0.000000
coworkers                      0  0.000000
mental_health_consequence      0  0.000000
gender                         0  0.000000
leave                          0  0.000000
anonymity                      0  0.000000
seek_help                      0  0.000000
wellness_program               0  0.000000
care_options                   0  0.000000
benefits   

* Replace NaNs with the second most frequently occuring factor in each feature

In [17]:
nan_list = data.columns.tolist()

for feature in nan_list:
    data[feature] = data[feature].fillna(list(data[feature].value_counts(dropna=False).to_dict().keys())[1])


In [18]:
data

,age,gender,self_employed,family_history,treatment,work_interfere,no_employees,remote_work,tech_company,benefits,...,mental_vs_physical,obs_consequence,race,mental_health,mental_health_diagnosed,obs_neg_response,physical_importance,mental_importance,neg_consequence_coworker,age_range
0,37,female,No,No,Yes,Often,Jun-25,No,Yes,Yes,...,Yes,No,White,Yes,Yes,No,5.0,5.0,No,31-65
1,44,male,No,No,No,Rarely,More than 1000,No,No,Don't know,...,Don't know,No,White,Yes,Yes,No,5.0,5.0,No,31-65
2,32,male,No,No,No,Rarely,Jun-25,No,Yes,No,...,No,No,White,Yes,Yes,No,5.0,5.0,No,31-65
3,31,male,No,Yes,Yes,Often,26-100,No,Yes,No,...,No,Yes,White,Yes,Yes,No,5.0,5.0,No,31-65
4,31,male,No,No,No,Never,100-500,Yes,Yes,Yes,...,Don't know,No,White,Yes,Yes,No,5.0,5.0,No,31-65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1917,33,male,No,I don't know,Yes,Sometimes,26-100,No,Yes,No,...,Physical health,No,White,Yes,Yes,"Yes, I experienced",4.0,1.0,No,31-65
1918,49,male,No,Yes,No,Not applicable to me,100-500,No,Yes,No,...,Physical health,No,White,No,Yes,No,8.0,5.0,No,31-65
1919,28,male,Yes,No,No,Not applicable to me,More than 1000,No,Yes,Yes,...,Don't know,No,White,No,Yes,I've always been self-employed,5.0,5.0,No,21-30
1920,26,male,Yes,No,No,Sometimes,More than 1000,No,Yes,Yes,...,Don't know,No,White,Don't Know,Yes,Maybe/Not sure,5.0,5.0,Yes,21-30


In [19]:
features = data.columns

counter = 1
for feature in features:
    print(f" {counter} ======== UNIQUE FACTORS IN {feature} ======")
    print(data[feature].unique())
    counter += 1
    print("\n")

 1 ======== UNIQUE FACTORS IN age ======
[37 44 32 31 33 35 39 42 23 29 36 27 46 41 34 30 40 38 50 24 18 28 26 22
 19 25 45 21 43 56 60 54 55 48 20 57 58 47 62 51 65 49 53 61 72 52 64 63
 59]


 2 ======== UNIQUE FACTORS IN gender ======
['female' 'male' 'trans']


 3 ======== UNIQUE FACTORS IN self_employed ======
['No' 'Yes']


 4 ======== UNIQUE FACTORS IN family_history ======
['No' 'Yes' "I don't know"]


 5 ======== UNIQUE FACTORS IN treatment ======
['Yes' 'No']


 6 ======== UNIQUE FACTORS IN work_interfere ======
['Often' 'Rarely' 'Never' 'Sometimes' 'Not applicable to me']


 7 ======== UNIQUE FACTORS IN no_employees ======
['Jun-25' 'More than 1000' '26-100' '100-500' '01-May' '500-1000' '6-25'
 '1-5']


 8 ======== UNIQUE FACTORS IN remote_work ======
['No' 'Yes']


 9 ======== UNIQUE FACTORS IN tech_company ======
['Yes' 'No']


 10 ======== UNIQUE FACTORS IN benefits ======
['Yes' "Don't know" 'No' "I don't know" 'Not eligible for coverage / NA']


 11 ======== UNIQUE FAC

# Encoding Data

In [20]:
#Encoding data
labelDict = {}
for feature in data:
    le = preprocessing.LabelEncoder()
    le.fit(data[feature])
    le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    data[feature] = le.transform(data[feature])
    
    
    # Get labels
    labelKey = 'label_' + feature
    labelValue = [*le_name_mapping]
    labelDict[labelKey] = labelValue
    
for key, value in labelDict.items():     
    print(key, value)

label_age [18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 72]
label_gender ['female', 'male', 'trans']
label_self_employed ['No', 'Yes']
label_family_history ["I don't know", 'No', 'Yes']
label_treatment ['No', 'Yes']
label_work_interfere ['Never', 'Not applicable to me', 'Often', 'Rarely', 'Sometimes']
label_no_employees ['01-May', '1-5', '100-500', '26-100', '500-1000', '6-25', 'Jun-25', 'More than 1000']
label_remote_work ['No', 'Yes']
label_tech_company ['No', 'Yes']
label_benefits ["Don't know", "I don't know", 'No', 'Not eligible for coverage / NA', 'Yes']
label_care_options ['No', 'Not sure', 'Yes']
label_wellness_program ["Don't know", "I don't know", 'No', 'Yes']
label_seek_help ["Don't know", "I don't know", 'No', 'Yes']
label_anonymity ["Don't know", "I don't know", 'No', 'Yes']
label_leave ['Difficult', "Don't know", "I don't know"

### Double-check for missing values after encoding data

In [21]:
total_missing = data.isnull().sum().sort_values(ascending=False)
percent = (data.isnull().sum()/data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total_missing, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)
print(missing_data)

                           Total  Percent
age                            0      0.0
phys_health_consequence        0      0.0
neg_consequence_coworker       0      0.0
mental_importance              0      0.0
physical_importance            0      0.0
obs_neg_response               0      0.0
mental_health_diagnosed        0      0.0
mental_health                  0      0.0
race                           0      0.0
obs_consequence                0      0.0
mental_vs_physical             0      0.0
phys_health_interview          0      0.0
mental_health_interview        0      0.0
supervisor                     0      0.0
coworkers                      0      0.0
mental_health_consequence      0      0.0
gender                         0      0.0
leave                          0      0.0
anonymity                      0      0.0
seek_help                      0      0.0
wellness_program               0      0.0
care_options                   0      0.0
benefits                       0  

## train-test split

In [22]:
X = data.drop('treatment', axis = 1)
y = data.treatment

In [23]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

## Finding the best model (using the complete dataset features)

* svc_full_feature
* bdt_full_feature
* gnb_full_feature

### Using SVC

In [24]:

svc_full_feature = SVC(kernel='rbf', probability=True, gamma="scale", C=1)
svc_full_feature.fit(X_train, y_train)

SVC(C=1, probability=True)

In [25]:
result_svc_full_feature = svc_full_feature.predict(X_test)
result_svc_full_feature

array([1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0,

In [26]:
a = pd.DataFrame({'Actual value': y_test, 'Predicted value': result_svc_full_feature})
a.head(10)

,Actual value,Predicted value
361,1,1
1552,0,0
1248,1,1
1633,1,0
427,0,1
294,1,1
161,1,1
825,0,1
182,0,0
223,1,1


In [27]:
#Confusion Matrix of SVC
svc_full_feature_cm = confusion_matrix(y_test, result_svc_full_feature)
print(svc_full_feature_cm )

[[130  64]
 [ 42 148]]


In [28]:
#Classification Report for SVC
print(classification_report(y_test, result_svc_full_feature))

              precision    recall  f1-score   support

           0       0.76      0.67      0.71       194
           1       0.70      0.78      0.74       190

    accuracy                           0.72       384
   macro avg       0.73      0.72      0.72       384
weighted avg       0.73      0.72      0.72       384



### Using Boosted Decision Tree

In [29]:
# the DT is boosted using AdaBoost

seed = np.random.RandomState(1)

dt_full_feature = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=5), n_estimators=500, random_state=seed
)

dt_full_feature.fit(X_train, y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=500,
                   random_state=RandomState(MT19937) at 0x136E5C79488)

In [30]:
result_dt_full_feature = dt_full_feature.predict(X_test)
result_dt_full_feature

array([1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0,
       1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0,

In [31]:
b = pd.DataFrame({'Actual value': y_test, 'Predicted value': result_dt_full_feature})
b.head(10)

,Actual value,Predicted value
361,1,1
1552,0,0
1248,1,1
1633,1,0
427,0,0
294,1,1
161,1,1
825,0,1
182,0,0
223,1,1


In [32]:
#Confusion Matrix of BDT
dt_full_feature_cm = confusion_matrix(y_test, result_dt_full_feature)
print(dt_full_feature_cm )

[[134  60]
 [ 51 139]]


In [33]:
#Classification Report for BDT
print(classification_report(y_test, result_dt_full_feature))

              precision    recall  f1-score   support

           0       0.72      0.69      0.71       194
           1       0.70      0.73      0.71       190

    accuracy                           0.71       384
   macro avg       0.71      0.71      0.71       384
weighted avg       0.71      0.71      0.71       384



### Using Gaussian Naive Bayes

In [34]:
gnb_full_feature = GaussianNB()
gnb_full_feature.fit(X_train, y_train)

GaussianNB()

In [35]:
result_gnb_full_feature = gnb_full_feature.predict(X_test)
result_gnb_full_feature

array([1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0,

In [36]:
c = pd.DataFrame({'Actual value': y_test, 'Predicted value': result_gnb_full_feature})
c.head(10)

,Actual value,Predicted value
361,1,1
1552,0,0
1248,1,1
1633,1,1
427,0,1
294,1,0
161,1,1
825,0,1
182,0,0
223,1,1


In [37]:
# Confusion Matrix of GNB
gnb_full_feature_cm = confusion_matrix(y_test, result_gnb_full_feature)
print(gnb_full_feature_cm)

[[140  54]
 [ 42 148]]


In [38]:
# Classification Report for GBN
print(classification_report(y_test, result_gnb_full_feature))

              precision    recall  f1-score   support

           0       0.77      0.72      0.74       194
           1       0.73      0.78      0.76       190

    accuracy                           0.75       384
   macro avg       0.75      0.75      0.75       384
weighted avg       0.75      0.75      0.75       384



# Actual values vs Predicted value by each model on the full features

In [42]:
compare_full = pd.DataFrame({'Actual': y_test, 'SVC': result_svc_full_feature, 'BDT': result_dt_full_feature, 'GNB': result_gnb_full_feature})

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

compare_full["summary"] = compare_full.sum(axis=1)

display(compare_full.head())

,Actual,SVC,BDT,GNB,summary
361,1,1,1,1,4
1552,0,0,0,0,0
1248,1,1,1,1,4
1633,1,0,0,1,2
427,0,1,0,1,2


In [43]:
summary = compare_full['summary'].tolist()
count = Counter(summary).most_common(2)
most_common = [paired for paired in count]
common_text = str(most_common)
common_text

'[(4, 110), (0, 105)]'

## Full features: Groups that are easier to classify by all 3 algos

In [45]:
table_class = pd.DataFrame()
table_class['True Positive'] = np.array([f"{str(round((110/384)*100))}%"])
table_class['True Negative'] = np.array([f"{str(round((105/384)*100))}%"])
table_class['Mixed classification'] = np.array([f"{str(round((169/384)*100))}%"])
table_class

,True Positive,True Negative,Mixed classification
0,29%,27%,44%


In [44]:
compare_full.shape

(384, 5)

In [ ]:
# Resets the options
pd.reset_option('all')

As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead.

: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.



In [ ]:
#compare_full.to_csv('actual_vs_models_pred_full.csv', index=True)

## Select eight best features from Mental Health data

In [ ]:
select_best = SelectKBest(score_func=chi2, k = 8)

In [ ]:
X_reduced = select_best.fit_transform(X, y)

In [ ]:
print("After selecting best 8 features:", X_reduced.shape)

After selecting best 8 features: (1920, 8)


In [ ]:
filter = select_best.get_support()
features = np.array(X.columns)
 

print("All features:")
print(features)
 
print('\n')
print('=============')

print("Selected best 8:")
print(features[filter])


All features:
['age' 'gender' 'self_employed' 'family_history' 'work_interfere'
 'no_employees' 'remote_work' 'tech_company' 'benefits' 'care_options'
 'wellness_program' 'seek_help' 'anonymity' 'leave'
 'mental_health_consequence' 'phys_health_consequence' 'coworkers'
 'supervisor' 'mental_health_interview' 'phys_health_interview'
 'mental_vs_physical' 'obs_consequence' 'race' 'mental_health'
 'mental_health_diagnosed' 'obs_neg_response' 'physical_importance'
 'mental_importance' 'neg_consequence_coworker' 'age_range']


Selected best 8:
['age' 'family_history' 'work_interfere' 'benefits' 'care_options'
 'anonymity' 'obs_consequence' 'mental_health']


In [ ]:
X_reduced

array([[19,  1,  2, ...,  3,  0,  3],
       [26,  1,  3, ...,  0,  0,  3],
       [14,  1,  3, ...,  0,  0,  3],
       ...,
       [10,  1,  1, ...,  0,  0,  1],
       [ 8,  1,  4, ...,  0,  0,  0],
       [20,  1,  4, ...,  2,  0,  3]], dtype=int64)

### train-test split for reduced features

In [ ]:
X_reduced_train, X_reduced_test, y_reduced_train, y_reduced_test = train_test_split(
                        X_reduced, y, test_size=0.2, random_state=0)

## Finding the best model (Using the reduced dataset features)

* svc_reduced_feature
* bdt_reduced_feature
* gnb_reduced_feature

### Using SVC

In [ ]:
svc_reduced_feature = SVC(kernel='rbf', probability=True, gamma="scale", C=1)
svc_reduced_feature.fit(X_reduced_train, y_reduced_train)

SVC(C=1, probability=True)

In [ ]:
result_svc_reduced_feature = svc_reduced_feature.predict(X_reduced_test)
result_svc_reduced_feature

array([1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0,

In [ ]:
a = pd.DataFrame({'Actual value': y_reduced_test, 'Predicted value': result_svc_reduced_feature})
a.head(10)

,Actual value,Predicted value
361,1,1
1552,0,0
1248,1,1
1633,1,0
427,0,1
294,1,1
161,1,1
825,0,0
182,0,0
223,1,1


In [ ]:
# Confusion Matrix of SVC
svc_reduced_feature_cm = confusion_matrix(y_reduced_test, result_svc_reduced_feature)
print(svc_reduced_feature_cm )

[[136  58]
 [ 46 144]]


In [ ]:
# Classification Report for SVC
print(classification_report(y_reduced_test, result_svc_reduced_feature))

              precision    recall  f1-score   support

           0       0.75      0.70      0.72       194
           1       0.71      0.76      0.73       190

    accuracy                           0.73       384
   macro avg       0.73      0.73      0.73       384
weighted avg       0.73      0.73      0.73       384



### Using Boosted Decision Tree

In [ ]:
# the DT is boosted using AdaBoost

seed = np.random.RandomState(1)

dt_reduced_feature = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=5), n_estimators=500, random_state=seed
)

dt_reduced_feature.fit(X_reduced_train, y_reduced_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                   n_estimators=500,
                   random_state=RandomState(MT19937) at 0x1247D610048)

In [ ]:
result_dt_reduced_feature = dt_reduced_feature.predict(X_reduced_test)
result_dt_reduced_feature

array([1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1,
       0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1,
       0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0,
       1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,

In [ ]:
b = pd.DataFrame({'Actual value': y_reduced_test, 'Predicted value': result_dt_reduced_feature})
b.head(10)

,Actual value,Predicted value
361,1,1
1552,0,0
1248,1,0
1633,1,1
427,0,0
294,1,0
161,1,1
825,0,1
182,0,0
223,1,0


In [ ]:
# Confusion Matrix of BDT
dt_reduced_feature_cm = confusion_matrix(y_reduced_test, result_dt_reduced_feature)
print(dt_reduced_feature_cm )

[[131  63]
 [ 58 132]]


In [ ]:
# Classification Report for BDT
print(classification_report(y_reduced_test, result_dt_reduced_feature))

              precision    recall  f1-score   support

           0       0.69      0.68      0.68       194
           1       0.68      0.69      0.69       190

    accuracy                           0.68       384
   macro avg       0.69      0.68      0.68       384
weighted avg       0.69      0.68      0.68       384



### Using Gaussian Naive Bayes

In [ ]:
gnb_reduced_feature = GaussianNB()
gnb_reduced_feature.fit(X_reduced_train, y_reduced_train)

GaussianNB()

In [ ]:
result_gnb_reduced_feature = gnb_reduced_feature.predict(X_reduced_test)
result_gnb_reduced_feature

array([1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0,

In [ ]:
c = pd.DataFrame({'Actual value': y_reduced_test, 'Predicted value': result_gnb_reduced_feature})
c.head(10)

,Actual value,Predicted value
361,1,1
1552,0,0
1248,1,1
1633,1,0
427,0,1
294,1,0
161,1,1
825,0,1
182,0,0
223,1,1


In [ ]:
# Confusion Matrix of GNB
gnb_reduced_feature_cm = confusion_matrix(y_reduced_test, result_gnb_reduced_feature)
print(gnb_reduced_feature_cm)

[[135  59]
 [ 37 153]]


In [ ]:
# Classification Report for GBN
print(classification_report(y_reduced_test, result_gnb_reduced_feature))

              precision    recall  f1-score   support

           0       0.78      0.70      0.74       194
           1       0.72      0.81      0.76       190

    accuracy                           0.75       384
   macro avg       0.75      0.75      0.75       384
weighted avg       0.75      0.75      0.75       384



In [ ]:
compare_reduced = pd.DataFrame({'Actual': y_reduced_test, 'SVC': result_svc_reduced_feature, 'BDT': result_dt_reduced_feature, 'GNB': result_gnb_reduced_feature})

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

compare_reduced["summary"] = compare_reduced.sum(axis=1)

display(compare_reduced.head())

,Actual,SVC,BDT,GNB,summary
361,1,1,1,1,4
1552,0,0,0,0,0
1248,1,1,0,1,3
1633,1,0,1,0,2
427,0,1,0,1,2


In [ ]:
summary = compare_reduced['summary'].tolist()
count = Counter(summary).most_common(2)
most_common = [paired for paired in count]
common_text = str(most_common)
common_text

'[(4, 107), (0, 100)]'

## Reduced features: Groups that are easier to classify by all 3 algos

In [ ]:
table_class = pd.DataFrame()
table_class['True Positive'] = np.array([f"{str(round((107/384)*100))}%"])
table_class['True Negative'] = np.array([f"{str(round((100/384)*100))}%"])
table_class['Mixed classification'] = np.array([f"{str(round((177/384)*100))}%"])
table_class

,True Positive,True Negative,Mixed classification
0,28%,26%,46%


In [ ]:
compare_reduced.shape

(384, 5)

In [ ]:
# Resets the options
pd.reset_option('all')

As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead.

: boolean
    use_inf_as_null had been deprecated and will be removed in a future
    version. Use `use_inf_as_na` instead.



In [ ]:
#compare_reduced.to_csv('actual_vs_models_pred_reduced.csv', index=True)